In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import requests
import time
import re
import os

In [2]:
def grab_all_urls_of_this_type(car_type):
    # Open the chrome browser
    driver = webdriver.Chrome('C:/Users/yangm/Desktop/chromedriver/chromedriver.exe')
    driver.get("http://www.edmunds.com/")
    driver.maximize_window()
    
    # Enter to the webpage with this car_type
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 600)")
    driver.find_element_by_css_selector("a[data-tracking-value='{}']".format(car_type)).click()
    # Close the ads
    time.sleep(1)
    driver.find_element_by_css_selector(".fixed-bottom.sticky-banner>.edm-entry.aunit.pos-r>button[data-tracking-id='close_adhesion']").click()
    
    # Find all collapsed elements
    collapse_buttons = driver.find_elements_by_css_selector(".collapse-control.mb-0.row.align-items-center.mx-0.mb-1_5.justify-content-start>button[data-tracking-id='subtype_list_expand']")
    
    # Click each of them
    for i in range(len(collapse_buttons)):
        ActionChains(driver).move_to_element(collapse_buttons[i]).perform()
        time.sleep(1)
        collapse_buttons[i].click()
                
    urls_and_years = driver.find_elements_by_css_selector(".vehicle-card.rounded.mb-1_5.pb-1")
    urls = []

    for i in range(len(urls_and_years)):
        url = urls_and_years[i].find_element_by_css_selector("a[data-tracking-value='model_image']").get_attribute("href")
        
        try:
            years = urls_and_years[i].find_element_by_css_selector(".mb-0.list-unstyled.col-12.col-md-8>.mb-1>.label").text[-4:]
            years = int(years)
            print(years)
            while years <= 2022:
                url = re.sub('[0-9]{4}',str(years),url)
                url = os.path.join(url + "consumer-reviews/?pagesize=50")   
                urls.append(url)
                years += 1             
        except:
            url = os.path.join(url + "consumer-reviews/?pagesize=50")
            urls.append(url)
            print('error')
            continue
            
    return urls

In [3]:
# driver = webdriver.Chrome('C:/Users/yangm/Desktop/chromedriver/chromedriver.exe')
# driver.get("http://www.edmunds.com/")
# driver.maximize_window()

# # Enter to the webpage with this car_type
# time.sleep(1)
# driver.execute_script("window.scrollTo(0, 600)")
# driver.find_element_by_css_selector("a[data-tracking-value='{}']".format('SUVs')).click()
# # Close the ads
# time.sleep(1)
# driver.find_element_by_css_selector(".fixed-bottom.sticky-banner>.edm-entry.aunit.pos-r>button[data-tracking-id='close_adhesion']").click()

# # Find all collapsed elements
# collapse_buttons = driver.find_elements_by_css_selector(".collapse-control.mb-0.row.align-items-center.mx-0.mb-1_5.justify-content-start>button[data-tracking-id='subtype_list_expand']")

# # Click each of them
# for i in range(len(collapse_buttons)):
#     ActionChains(driver).move_to_element(collapse_buttons[i]).perform()
#     time.sleep(1)
#     collapse_buttons[i].click()

In [4]:
# earlist_model_years = driver.find_elements_by_css_selector(".mb-0.list-unstyled.col-12.col-md-8>.mb-1>.label")
# url_elements = driver.find_elements_by_css_selector("a[data-tracking-value='model_image']")
# for i in range(len(url_elements)):
#     print(earlist_model_years[i].text)
#     print(url_elements[i].get_attribute("href"))
#     print("---------------------")
    

In [5]:
def get_posts(url,count=False):
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
    }

    response = requests.get(url, timeout=10, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')
    if count:
        posts = soup.find('div', class_ ='review-count text-gray-darker')
    else:
        posts = soup.find_all('div', class_ ='review-item text-gray-darker')

    return posts

In [6]:
def find_review_and_ratings_of_this_page(posts):
    # Create all lists
    reviews = []
    Total_star = []
    Safety = []
    Technology = []
    Performance = []
    Interior = []
    Comfort = []
    Reliability = []
    Value = []
    
    for i in range(len(posts)):
        # Reviews and total stars
        reviews.append(posts[i].find('p').text.strip())
        Total_star.append(posts[i].find('span',class_="sr-only").text[0])

        # Safety rating star (if none, output 0)
        Safety_tag = posts[i].find('dt',string="Safety")
        if Safety_tag is None:
            Safety.append('0')
        else:
            Safety.append(Safety_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Technology rating star (if none, output 0)
        Technology_tag = posts[i].find('dt',string="Technology")
        if Technology_tag is None:
            Technology.append('0')
        else:
            Technology.append(Technology_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Performance rating star (if none, output 0)
        Performance_tag = posts[i].find('dt',string="Performance")
        if Performance_tag is None:
            Performance.append('0')
        else:
            Performance.append(Performance_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Interior rating star (if none, output 0)
        Interior_tag = posts[i].find('dt',string="Interior")
        if Interior_tag is None:
            Interior.append('0')
        else:
            Interior.append(Interior_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Comfort rating star (if none, output 0)
        Comfort_tag = posts[i].find('dt',string="Comfort")
        if Comfort_tag is None:
            Comfort.append('0')
        else:
            Comfort.append(Comfort_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Reliability rating star (if none, output 0)
        Reliability_tag = posts[i].find('dt',string="Reliability")
        if Reliability_tag is None:
            Reliability.append('0')
        else:
            Reliability.append(Reliability_tag.next_sibling.find('span',class_='sr-only').text[0])

        # Value rating star (if none, output 0)
        Value_tag = posts[i].find('dt',string="Value")
        if Value_tag is None:
            Value.append('0')
        else:
            Value.append(Value_tag.next_sibling.find('span',class_='sr-only').text[0])
            
    df = pd.DataFrame({'Total_star': Total_star,
         'Review':reviews,
         'Safety': Safety,
         'Technology': Technology,
         'Performance': Performance,
         'Interior': Interior,
         'Comfort': Comfort,
         'Reliability': Reliability,
         'Value': Value,
    })

    return df

In [7]:
def the_number_of_pages(post):

    number_of_reviews = int(post.text[:2])
    
    number_of_pages = number_of_reviews//50
    if number_of_reviews%50 > 0:
        number_of_pages += 1

    return number_of_pages

In [8]:
def grab_reviews_and_ratings_of_this_model(url):
    count_post = get_posts(url,count=True)
    number_of_pages = the_number_of_pages(count_post)
    df = pd.DataFrame()
    
    for page_number in range(number_of_pages):
        page_number += 1
        url = os.path.join(url + "&pagenum=%s"%page_number)
        posts = get_posts(url)
        temp_df = find_review_and_ratings_of_this_page(posts)
        df = pd.concat([df,temp_df],ignore_index = True)
        
    # Obtain the basic information
    url_splited = re.split('/',url)
    make = url_splited[3]
    df.insert(0,'Make',make)
    model = url_splited[4]
    df.insert(1,'Model',model)
    year = url_splited[5]
    df.insert(2,'Year',year)
    
    return df

In [9]:
car_types = ['SUVs', 'Trucks','Sedans','Electric Cars','Hybrids','Hatchbacks','Coupes','Wagons','Convertibles','Minivans']

In [10]:
error = 0
errors = []
list_of_dfs = []
for car_type in car_types:
    dfs = pd.DataFrame()
    urls = grab_all_urls_of_this_type(car_type)
    for url in urls:
        try:
            df_temp = grab_reviews_and_ratings_of_this_model(url)
            dfs = pd.concat([dfs,df_temp],ignore_index = True)
        except:
            error += 1
            errors.append(url)
            
    print(error)
    list_of_dfs.append(dfs)

2022
2021
2020
2020
2018
2018
2020
2016
2021
2016
2017
2017
2021
2022
2019
2014
2018
2019
2020
2021
2021
2022
2018
2019
2020
2021
2019
2020
2015
2010
2011
2019
2015
2020
2020
2016
2018
2016
2020
2022
2019
2011
2010
2021
2021
2018
2021
2008
2017
2008
2020
2016
2021
2019
2019
2018
2019
2019
2016
2018
2019
2015
2018
2020
2018
2019
2018
2021
2020
2021
2019
2019
2021
2020
2019
2016
2020
2017
2017
2022
2020
2017
2018
2020
2014
2010
2020
2018
2021
2019
2008
2021
error
2019
2013
2017
2020
2020
2018
2018
2018
2019
2020
2020
2011
2020
2011
2020
2021
2017
2021
120
2017
2020
2016
2015
2015
2022
2019
2021
2019
2019
2019
2017
2007
2019
2017
2020
149
2019
2018
2020
2022
2019
2021
2019
2017
2020
2019
2020
2021
2018
2014
2018
2020
2018
2018
2019
2020
2016
2019
2019
2011
2017
2011
2019
2019
2020
2014
2015
2019
2021
2017
2015
2017
2017
2019
2019
2019
2017
2017
2021
2019
2020
2013
2014
2016
2020
2017
2017
2018
2019
2016
2017
error
2017
2016
2018
2018
2020
2020
2021
2018
2021
2020
2014
2020
2020
2017
2016


In [11]:
final_df = pd.DataFrame()
for dfs,car_type in zip(list_of_dfs,car_types):
    dfs.insert(0,"Type",car_type)
    final_df = pd.concat([final_df,dfs])

In [12]:
final_df

,Type,Make,Model,Year,Total_star,Review,Safety,Technology,Performance,Interior,Comfort,Reliability,Value
0,SUVs,volkswagen,taos,2022,5,"I recently traded in my 2017 Honda HR-V in ""Ba...",5,4,4,4,5,5,5
1,SUVs,volkswagen,taos,2022,4,"Just to be clear, I don’t own a Taos, but I am...",4,4,4,4,4,4,4
2,SUVs,volkswagen,taos,2022,5,Recently purchased a Taos in the base trim (S)...,5,5,5,3,4,0,5
3,SUVs,volkswagen,taos,2022,5,This car feels premium and looks handsome. It...,0,5,5,5,5,0,5
4,SUVs,volkswagen,taos,2022,1,"Bought the White SEL, love everything about it...",1,3,3,4,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,Minivans,kia,carnival,2022,5,My wife and I recently retired and wanted a ne...,5,5,5,5,5,5,5
342,Minivans,kia,carnival,2022,5,Just bought one yesterday in the SX trim and i...,5,5,5,5,5,5,5
343,Minivans,kia,carnival,2022,5,It is great. Enough room for everyone plus car...,5,5,5,5,5,5,5
344,Minivans,kia,carnival,2022,3,Other than UVO not available in my state is th...,5,5,5,5,5,5,3


In [13]:
final_df.head()

,Type,Make,Model,Year,Total_star,Review,Safety,Technology,Performance,Interior,Comfort,Reliability,Value
0,SUVs,volkswagen,taos,2022,5,"I recently traded in my 2017 Honda HR-V in ""Ba...",5,4,4,4,5,5,5
1,SUVs,volkswagen,taos,2022,4,"Just to be clear, I don’t own a Taos, but I am...",4,4,4,4,4,4,4
2,SUVs,volkswagen,taos,2022,5,Recently purchased a Taos in the base trim (S)...,5,5,5,3,4,0,5
3,SUVs,volkswagen,taos,2022,5,This car feels premium and looks handsome. It...,0,5,5,5,5,0,5
4,SUVs,volkswagen,taos,2022,1,"Bought the White SEL, love everything about it...",1,3,3,4,4,1,2


In [17]:
final_df["Type"].value_counts()

SUVs             13850
Sedans            4632
Trucks            2647
Hybrids           2636
Coupes            1459
Electric Cars     1266
Hatchbacks         429
Minivans           346
Convertibles       327
Wagons              55
Name: Type, dtype: int64

In [16]:
final_df.to_csv('inital_train_data.csv')